In [ ]:
!pip install transformers
!pip install tensorflow_addons

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
     |████████████████████████████████| 52 kB 538 kB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 636 kB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFAutoModel, AlbertTokenizerFast
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint

drive_loc = './drive/MyDrive/'

#Load Data
with open(f'{drive_loc}Word_Arrays_Article_Altered.pkl', 'rb') as f:
    training, testing = pickle.load(f)

seq_len = 50 #512
batchs = 64

model_name = 'bert-base-cased'
#model_name = 'albert-base-v2'

tokenizer = BertTokenizer.from_pretrained(model_name)
#tokenizer = AlbertTokenizerFast.from_pretrained(model_name)
ori_test = testing[:, 1]

In [ ]:
training_len = len(training)
dataset = np.vstack((training, testing))
dataset[:, 1] = dataset[:, 1].astype(str)

In [ ]:
lister = []
for string in dataset[:, 0]:
  lenner = len(string.split())
  lister.append(lenner+1)

In [ ]:
np.array(lister).std()

413.3462714169685

In [ ]:
_input_ = training[-1,0]
_tok_ =  tokenizer(_input_, 
                      max_length = seq_len, 
                      truncation=True,
                      padding='max_length', 
                      add_special_tokens=True,
                      return_tensors='np'
                      )

In [ ]:
def data_tokenised(data, tok, seq_len):
    
    samples = len(data)
    num_labels = data[:, 1].astype(int).max()
    
    x_ids = np.zeros((samples, seq_len))
    x_mask = np.zeros((samples, seq_len))
    
    for i in range(len(data)):
        _input_ = data[i, 0]
        _tok_ =  tok(_input_, 
                      max_length = seq_len, 
                      truncation=True,
                      padding='max_length', 
                      add_special_tokens=True,
                      return_tensors='np'
                      )
        x_ids[i, :] = _tok_['input_ids']
        x_mask[i, :] = _tok_['attention_mask']
    
    lab = np.zeros((samples, num_labels+1))
    
    lab[np.arange(samples), data[:, 1].astype(int)] = 1
    lab = lab[:, 1].reshape(-1, 1)
    
    return x_ids, x_mask, lab

dataset = data_tokenised(dataset, tokenizer, seq_len)

In [ ]:
training = dataset[0][:training_len], dataset[1][:training_len], dataset[2][:training_len]
testing = dataset[0][training_len:], dataset[1][training_len:], dataset[2][training_len:]

In [ ]:
X = {'input_ids':training[0],
     'attention_mask':training[1]}
y = training[2]

X_test = {'input_ids':testing[0],
          'attention_mask':testing[1]}
y_test = testing[2]

In [ ]:

checkpoint_dir = f'{drive_loc}/Headline_Weights'
#checkpoint_dir = './Checkpoints/weights'

checkpoint = ModelCheckpoint(checkpoint_dir, monitor='val_accuracy', verbose=0, save_best_only=True, mode='auto')#, save_weights_only=True, mode='auto')  
callbacks = [checkpoint]

def create_model():
  BERT = TFAutoModel.from_pretrained(model_name)

  input_ids = tf.keras.layers.Input(shape=(seq_len, ), name='input_ids', dtype='int32')
  mask = tf.keras.layers.Input(shape=(seq_len, ), name='attention_mask', dtype='int32')

  embeddings = BERT.bert(input_ids, attention_mask=mask)['pooler_output'] #Try out index 0
  #embeddings = BERT.albert(input_ids, attention_mask=mask)['pooler_output']

  den_1 = tf.keras.layers.Dense(1000, activation='relu')(embeddings)
  #den_2  = tf.keras.layers.Dense(1000, activation='relu')(den_1)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(den_1)

  model = tf.keras.Model(inputs=[input_ids, mask], outputs=output)

  #opt = tfa.optimizers.AdamW()
  opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)
  loss = tf.keras.losses.BinaryCrossentropy()


  model.compile(opt, loss=loss, metrics=['accuracy'])
  #model.layers[0].trainable = False
  #model.layers[1].trainable = False
  #model.layers[2].trainable = False

  return model


In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

# Create model
with strategy.scope():
    model = create_model()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.58.31.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.58.31.58:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 50)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1000)         769000      bert[0][1]                   

In [ ]:
"""
history = model.fit(X, y,
                    validation_data = (X_test, y_test),
                    epochs = 1,
                    batch_size = batchs,
                    #callbacks = callbacks,
                    verbose=1
                    )"""

'\nhistory = model.fit(X, y,\n                    validation_data = (X_test, y_test),\n                    epochs = 1,\n                    batch_size = batchs,\n                    #callbacks = callbacks,\n                    verbose=1\n                    )'

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

epochs = 50
histories = {}

best_results = 0

for i in range(epochs):
  print(f'Epoch {i}')
  history = model.fit(X, y,
                      validation_data = (X_test, y_test),
                      epochs = 1,
                      batch_size = batchs,
                      verbose=1
                      )
  hist = history.history
  for key in hist.keys():
      if key not in histories:
        histories[key] = []
      else:
        histories[key].append(hist[key][0])
  
  val_acc = hist['val_accuracy'][0]

  if val_acc > best_results:
    print('New Best')
    preds = model.predict(X_test)

    preds[preds>0.5] = 1
    preds[preds<=0.5] = 0

    best_results = val_acc

    accuracy = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test,preds)

    

  

Epoch 0


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 1) dtype=float32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 1) dtype=float32>]


109/109 [==============================] - ETA: 0s - loss: 0.4058 - accuracy: 0.8106

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 1) dtype=float32>]


109/109 [==============================] - 157s 679ms/step - loss: 0.4058 - accuracy: 0.8106 - val_loss: 0.2398 - val_accuracy: 0.9112
New Best


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=float32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 50) dtype=float32>]


Epoch 1
109/109 [==============================] - 9s 86ms/step - loss: 0.2239 - accuracy: 0.9143 - val_loss: 0.2757 - val_accuracy: 0.8912
Epoch 2
109/109 [==============================] - 9s 86ms/step - loss: 0.1629 - accuracy: 0.9372 - val_loss: 0.2220 - val_accuracy: 0.9190
New Best
Epoch 3
109/109 [==============================] - 9s 86ms/step - loss: 0.1041 - accuracy: 0.9620 - val_loss: 0.2503 - val_accuracy: 0.9285
New Best
Epoch 4
109/109 [==============================] - 9s 87ms/step - loss: 0.0674 - accuracy: 0.9776 - val_loss: 0.2686 - val_accuracy: 0.9224
Epoch 5
109/109 [==============================] - 9s 86ms/step - loss: 0.0458 - accuracy: 0.9857 - val_loss: 0.3238 - val_accuracy: 0.9250
Epoch 6
109/109 [==============================] - 9s 87ms/step - loss: 0.0372 - accuracy: 0.9889 - val_loss: 0.3091 - val_accuracy: 0.9289
New Best
Epoch 7
109/109 [==============================] - 9s 86ms/step - loss: 0.0340 - accuracy: 0.9900 - val_loss: 0.3093 - val_accuracy: 

In [ ]:
#Results
#hist = model.history

histories['Metrics'] = {'Accuracy':accuracy,
                        'Precision_Score':prec,
                        'Recall':recall,
                        'f1_score':f1}


#with open(f'{drive_loc}/History-Headlines-Final.pkl', 'wb') as f:
    #pickle.dump(histories, f)
